# Animated Multicos Example

## Introduction

This notebook demonstrates Bayes updates and resampling steps of the sequential Monte Carlo algorithm, using [QInfer](https://github.com/csferrie/python-qinfer).

## Preamble

As always for Python 2, we need to enable sensible division support.

In [1]:
from __future__ import division

We want to [suppress output of figures](http://stackoverflow.com/questions/15713279/calling-pylab-savefig-without-display-in-ipython), or else we'll generate a lot of excess figures.

In [2]:
import matplotlib
matplotlib.use('Agg')

Now we can import the basic scientific and plotting libraries we need.

In [3]:
import numpy as np
import matplotlib.pyplot as plt

/home/cgranade/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Because I think it's prettier, we enable the ggplot style used by mpltools.

In [4]:
plt.style.use('ggplot-rq')
plt.rcParams['figure.facecolor'] = 'black'
plt.rcParams['text.color'] = 'white'
plt.rcParams['grid.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['axes.facecolor'] = '#444444'
plt.rcParams['axes.prop_cycle'] = plt.cycler('color', [
    '#D55E00', 
    '#56B4E9'
])

We can now import QInfer and the [multicos model](http://python-qinfer.readthedocs.org/en/latest/guide/models.html#implementing-custom-simulators-and-models) we'll be using.

In [5]:
import qinfer as qi
from multicos import MultiCosModel

Finally, we need some annoying OS stuff to put the figures in a nice temporary location.

In [6]:
import tempfile as tf
import os

In [7]:
save_loc = tf.mkdtemp(prefix='multicos-figs-')

In [8]:
print "Saving figures to {}".format(save_loc)

Saving figures to /tmp/multicos-figs-m9NTas


In [9]:
n_exps = 250

## Functions

We define two functions, one which saves out figures to PNGs for later use, and one which samples experiments.

In [10]:
def make_updater_frame(updater, fname=None, pt_size=12, true=None):
    locs = updater.particle_locations
    ws   = updater.particle_weights * updater.n_particles
    
    fig = plt.figure()
    plt.scatter(locs[:, 0], locs[:, 1], s=pt_size*np.sqrt(ws), c='#56B4E9', linewidth=0)
    
    if true is not None:
        plt.scatter(true[..., 0], true[..., 1], s=pt_size*10, c='#D55E00', marker='*')
    
    plt.xlim((0, 1))
    plt.ylim((0, 1))
    plt.axes().set_aspect('equal')
    plt.xlabel(r'$\omega_1$')
    plt.ylabel(r'$\omega_2$')
    plt.gca().grid(True)
#     plt.title(r'$\Pr(1 | \omega_1, \omega_2; t) = \cos^2(\omega_1 t_1) \cos^2(\omega_2 t_2)$', loc='left', y=1.2)
    legend = plt.legend(['Particles', 'True'], ncol=2, bbox_to_anchor=(1, 1.075), scatterpoints=1)
    
    # See http://stackoverflow.com/questions/24706125/setting-a-fixed-size-for-points-in-legend for why this works.
    legend.legendHandles[0]._sizes = [12.]
    
    if fname is not None:
        plt.savefig(os.path.join(save_loc, fname), format='png', dpi=300, facecolor='k', frameon=False)
        plt.close()
        del fig
        return None
    else:
        return fig

In [11]:
def heuristic(updater):
    idx_exp = len(updater.data_record)
    mag_t = (np.pi / 2) * ((129/128)**idx_exp)
    theta = np.random.rand() * np.pi / 2
    
    expparams = np.array([([np.cos(theta) * mag_t, np.sin(theta) * mag_t], )], dtype=model.expparams_dtype)
    return expparams

## The Actual Experiment

Let's now draw data and plot the posteriors!

In [12]:
prior = qi.UniformDistribution([[0, 1], [0, 1]])
true_model = prior.sample()

In [13]:
model = MultiCosModel()

In [14]:
updater = qi.smc.SMCUpdater(model, 1000, prior)

In [15]:
prog = qi.IPythonProgressBar()
prog.start(n_exps)

for idx_exp in xrange(n_exps):
    make_updater_frame(updater, true=true_model, fname='dist-{:04}.png'.format(idx_exp))
    expparams = heuristic(updater)
    outcome = model.simulate_experiment(true_model, expparams)
    updater.update(outcome, expparams)
    prog.update(idx_exp)
    
make_updater_frame(updater, true=true_model, fname='dist-{:04}.png'.format(n_exps))
prog.finished()

## Make The Video (*nix Only)

In this section, we [assemble the PNGs](https://trac.ffmpeg.org/wiki/Create%20a%20video%20slideshow%20from%20images) we made earlier into an MPEG-4 AVC (that is, H.264) container. We assume [avconv, not ffmpeg](http://askubuntu.com/a/432585) on an Ubuntu-like system.

In [16]:
cmd = r'avconv -framerate 10 -i {0}/dist-%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p {0}/distns.mp4'.format(save_loc)

In [17]:
import subprocess

WARNING: the following can take a long time, and if it fails, lock up IPython Notebook. Use with caution.

In [18]:
retval = subprocess.call(cmd, shell=True)
if not retval:
    subprocess.call('mv {0}/distns.mp4 figures/multicos-distributions.mp4'.format(save_loc), shell=True)